In [ ]:
%load_ext autoreload
%autoreload 2

# 2023-02-28 Trying to get Surface Plots Working

In [ ]:
import os

os.getcwd()

## Load the required software and data

In [ ]:
from pathlib import Path


from scripts.data_interface import retrieve_uv_data

import rainbow as rb

p = Path("/Users/jonathan/0_jono_data/2023-02-22_2021-DEBORTOLI-CABERNET-MERLOT_HALO.D")

print(p)

uv_data = retrieve_uv_data(rb.read(str(p)))

## Trying with `Extend_Traces()`

lets use a known dataset - `2023-02-22_2021-DEBORTOLI-CABERNET-MERLOT_HALO.D`. A quick description:

## Preparing the Data

In [ ]:
uv_data.describe()

Anything with a max abs that is too high or too low can be discarded. To do this, we can define a function that can take a dimension as a target, then set min and max values:

#### Defining a Windowing Function

In [ ]:
def df_windower(dimension=str, min_val=int, max_val=int):
    melt_d = uv_data.melt(id_vars="mins", value_name="mAU", var_name="nm")

    melt_d = melt_d[
        (melt_d[f"{dimension}"] > min_val) & (melt_d[f"{dimension}"] < max_val)
    ]

    pivot_d = melt_d.pivot(columns=["nm"], values="mAU", index="mins")

    pivot_d = pivot_d.reset_index()

    return pivot_d

And test it with 'nm':

In [ ]:
window_uv_data = df_windower("nm", 211, 255)

window_uv_data.describe()

How about 'mins'?

In [ ]:
window_uv_data = df_windower("mins", 12, 24)

window_uv_data.describe()

And wb absorbance?

In [ ]:
window_uv_data = df_windower("mAU", 500, 1000)

window_uv_data.describe()

Doesn't quite work for absorbance because itll drop any entry where the absorbance is outside of the window. Good for the other two though.

### Producing a Surface

In [ ]:
import math

import numpy as np


def square_surface(data_array):
    print("calculating array size to be squarable:")
    try:
        next_sqrt = 1 + math.isqrt(data_array.size - 1)
        print(f"next highest integer root is {next_sqrt}")

    except Exception as e:
        print(f"failed to calculate array size: f{e}")

    print(f"resizing array with size {next_sqrt**2} by {next_sqrt**2}")

    try:
        resized_array = np.resize(data_array, (next_sqrt**2, next_sqrt**2))

        print(f"new array is size {resized_array.size}")

    except Exception as e:
        print(f"failed to resize the array: {e}")

    dim = np.sqrt(resized_array.size).astype(int)

    print(
        f"to from a square, the array will have sides of length {dim}. Reshaping now:"
    )

    try:
        square_array = resized_array.reshape(dim, dim)
    except Exception as e:
        print(f"Failed to reshape the array: {e}")

    return square_array

In [ ]:
Z = square_surface(pivot_d)

Z.size

11-04: The dataset between 211 and 255nm is still way to big without any sort of processing. I need to look into smoothing and downsampling techniques before progressing further.

## Data Streaming

The following cell is a template for data streaming a set into a 3d surface dynamically, generated by ChatGPT.

In [ ]:
# import plotly.graph_objs as go
# import time

# fig = go.Figure(data=[go.Surface(z=[])])

# fig.show()

# num_batchs = 50

# for i in range(num_batches):
#     # Generate new batch of data
#     x_new, y_new, z_new = generate_new_data()

#     # Update the data of the surface plot using extend_traces
#     fig.data[0].extend_traces(
#         x=[x_new], y=[y_new], z=[z_new],
#         indices=[(i, j) for i in range(len(x_new)) for j in range(len(y_new))]
#     )

#     # Wait for a short time before generating the next batch of data
#     time.sleep(0.5)

# fig.update_layout(title='Surface plot with data streaming')
# fig.show()